In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%config Completer.use_jedi = False
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)




#ploting imports
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ubiquant-market-prediction/example_sample_submission.csv
/kaggle/input/ubiquant-market-prediction/example_test.csv
/kaggle/input/ubiquant-market-prediction/train.csv
/kaggle/input/ubiquant-market-prediction/ubiquant/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/ubiquant-market-prediction/ubiquant/__init__.py


In [2]:
#column reduction
basic_cols=['target']
test_cols=['row_id','investment_id']

num_feat=75
features=[f'f_{i}' for i in range(num_feat)]
cols=basic_cols+features
train_cols=test_cols+test_cols


In [3]:
#load data
train=pd.read_csv('../input/ubiquant-market-prediction/train.csv',usecols=cols)
test_df=pd.read_csv("../input/ubiquant-market-prediction/example_test.csv",usecols=features)
sample_prediction_df=pd.read_csv('../input/ubiquant-market-prediction/example_sample_submission.csv')
print("data is loaded!!!!")

data is loaded!!!!


In [4]:
print("train shape is {}".format(train.shape))
print("example_test shape is{}".format(test_df.shape))
print("example_sample_submission shape is{}".format(sample_prediction_df.shape))

train shape is (3141410, 76)
example_test shape is(9, 75)
example_sample_submission shape is(9, 3)


In [5]:
train.head()

,target,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74
0,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,0.965623,1.230508,0.114809,...,0.662736,1.933138,-0.812392,0.072803,-0.778987,-0.553666,-0.032761,-0.619364,-1.424479,0.148335
1,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,1.428127,1.134144,0.114809,...,-0.292549,-0.550049,-1.629836,0.902852,0.509950,0.377552,0.052099,-0.763414,-1.307945,0.148335
2,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,0.979656,-1.125681,0.114809,...,0.309863,-0.550049,1.638620,-0.476601,-0.909049,-0.544951,-0.546394,0.163453,1.748066,0.148335
3,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,0.778096,0.298990,0.114809,...,0.464784,-0.550049,-0.831141,-0.653802,-0.941017,-0.424802,-0.413291,1.553026,-0.490458,0.148335
4,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,-0.946789,1.230508,0.114809,...,-1.240730,-0.550049,-0.812392,-0.696176,-0.410776,-0.783214,-0.695970,1.394426,-1.424479,0.148335


In [6]:
#missing value count
missing_value=train.isnull().sum()
print(missing_value)
total_cells=np.product(train.shape)
total_missing=missing_value.sum()
print("% of missing data=",(total_missing/total_cells))

target    0
f_0       0
f_1       0
f_2       0
f_3       0
         ..
f_70      0
f_71      0
f_72      0
f_73      0
f_74      0
Length: 76, dtype: int64
% of missing data= 0.0


In [7]:
X_train=train.loc[:, features]
y_train=train.loc[:, 'target']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=0.1,random_state=0)


In [9]:
regressor=xgb.XGBRegressor(
    tree_learner='serial',
    n_estimators=1000,
    num_leaves=64,
    max_depth=8,
    learning_rate=0.1,
    feature_fraction=0.6,
    subsample=0,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=2022,
    tree_method='gpu_hist')
regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=1)


[11:05:09] WARNING: ../src/learner.cc:576: 
Parameters: { "feature_fraction", "num_leaves", "tree_learner" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1.05763
[1]	validation_0-rmse:1.05763
[2]	validation_0-rmse:1.05763
[3]	validation_0-rmse:1.05763
[4]	validation_0-rmse:1.05763
[5]	validation_0-rmse:1.05763


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             feature_fraction=0.6, gamma=0, gpu_id=0, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=2,
             num_leaves=64, num_parallel_tree=1, predictor='auto',
             random_state=2022, reg_alpha=0.1, reg_lambda=0.1,
             scale_pos_weight=1, subsample=0, tree_learner='serial',
             tree_method='gpu_hist', validate_parameters=1, ...)

In [10]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    preds = regressor.predict(test_df[features].values)
    sample_prediction_df['target'] = preds
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
